In [1]:
# web automation
from selenium import webdriver
from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager

# web scraping
from bs4 import BeautifulSoup
import requests
# import json
import pandas as pd

import re

### Get project names w/ selenium

In [2]:
base_url = 'https://berkeleyca.gov/your-government/our-work/capital-projects'

In [7]:
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
driver.get(base_url)

In [ ]:
# get page navigation element links
# buttons = [elt.find_element(By.TAG_NAME,'a').get_attribute('href') for elt in driver.find_element(By.XPATH,'//*[@id="main-content"]/div/div/div/div/nav/ul').find_elements(By.TAG_NAME,'li')]
# buttons

In [9]:
driver.get(base_url + '?page=' + '4')

In [8]:
# for each page
# get links on all 6 pages
links = []
for i in range(0,6):
    driver.get(base_url + '?page=' + str(i))
    project_cells = driver.find_elements(By.TAG_NAME, 'tr')
    # append to list of links
    links = links + [cell.find_elements(By.TAG_NAME, 'td')[0].find_element(By.TAG_NAME,'a').get_attribute('href') for cell in project_cells if cell.find_elements(By.TAG_NAME, 'td')]
    # go to next page
links[-5:]

['https://berkeleyca.gov/your-government/our-work/capital-projects/university-west-bus-stops',
 'https://berkeleyca.gov/your-government/our-work/capital-projects/waterfront-specific-plan',
 'https://berkeleyca.gov/your-government/our-work/capital-projects/willard-clubhouse-project',
 'https://berkeleyca.gov/your-government/our-work/capital-projects/woolsey-street-low-impact-development-green',
 'https://berkeleyca.gov/your-government/our-work/capital-projects/woolsey-streeteton-avenue-intersection-project']

In [ ]:
pd.DataFrame(columns=['links'],data=links).to_csv('links.csv',index=False)

### Get individual project details from page w/ BS

In [12]:
len(links)

105

In [18]:
# for each link in list get following data
# name, company, status, type, image, description...
data_list = []
for i, link in enumerate(links):
    if i % 10 == 0:
        print(i, link)
    # go to page
    try:
        page = requests.get(link)
        soup = BeautifulSoup(page.content, "html.parser")

        # get info
        name = soup.find('h1').text
        content= soup.find('div',id='main-content')
        type_of_construction, status = [elt.find('span').text.strip('\n') for elt in content.find('div',class_='meta').find_all('div',class_='field-wrapper medium-shrink cell')]
        location = content.find('section',class_='block-field-blocknodecapital-projectfield-project-location').find('p').text.replace(u'\xa0', u'')
        description = ' '.join([p.text for p in content.find('section',class_='block-field-blocknodecapital-projectbody').find_all('p')]).replace(u'\xa0', u'')
        image = 'https://berkeleyca.gov/' + content.find('div',id=lambda x: x and x.startswith('slick-node-capital-project-images-custom-')).find('a').get('href') # using partial id to grab image div tag
        '//*[@id="slick-node-capital-project-images-custom-15901-2"]/a/div/img'
        # could consider getting project timeline/elements for some completed projects...
        if content.find('div',class_='project-funding'):
            funding_source = [l.text.strip() for l in content.find('div',class_='project-funding').find('ul').find_all('li')]
        else:
            funding_source = []
        project_team_info = [re.split('\n+\t*',elt.text.strip())[:3] for elt in content.find_all('div',class_='team-member-content')]
        project_team = [t[0] for t in project_team_info]

        # add to row for project
        project_row = [name,status,type_of_construction,location,description,funding_source,project_team,project_team_info,image]
        data_list.append(project_row)
    except:
        print('err',i, link)
        break
    
data_list

0 https://berkeleyca.gov/your-government/our-work/capital-projects/1740-san-pablo-avenue
10 https://berkeleyca.gov/your-government/our-work/capital-projects/berkeley-marina-roadway-improvements-project
20 https://berkeleyca.gov/your-government/our-work/capital-projects/city-wide-irrigation-system
30 https://berkeleyca.gov/your-government/our-work/capital-projects/frances-albrier-community-center-replacement-project
40 https://berkeleyca.gov/your-government/our-work/capital-projects/james-kenney-park-skate-spot
50 https://berkeleyca.gov/your-government/our-work/capital-projects/maudelle-miller-shirek-community
60 https://berkeleyca.gov/your-government/our-work/capital-projects/parker-addison-bike-boulevard
70 https://berkeleyca.gov/your-government/our-work/capital-projects/sacramento-complete-streets-project
80 https://berkeleyca.gov/your-government/our-work/capital-projects/shattuck-reconfiguration-and-pedestrian-safety-project
90 https://berkeleyca.gov/your-government/our-work/capital

[['1740 San Pablo Avenue',
  'Design',
  'Affordable Housing',
  '1740 San Pablo Avenue',
  'Transform unutilized space at 1740 San Pablo into 54 new affordable homes for Berkeley families. Groundbreaking is anticipated on 2023.',
  ['Measure O'],
  ['Jenny Wyant', 'Bridge Housing'],
  [['Jenny Wyant',
    'Senior Community Development Project Coordinator',
    'Health, Housing, and Community Services'],
   ['Bridge Housing', '(415) 983-1111']],
  'https://berkeleyca.gov//sites/default/files/2022-10/1740%20SPA%20rendering.png'],
 ['1947 Center Street Facility Improvements',
  'Design',
  'Facility',
  '1947 Center Street',
  'This is a Measure T1 Phase 2 project. The 1947 Center Street is a multi-purpose facility that provides services and information to those who live, work and visit the City of Berkeley. The building will undergo HVAC system improvements to ensure proper indoor air quality. This project is in the design phase. Most recently, a Testing, Adjusting, Balancing (TAB) stud

In [19]:
# convert list to dataframe and save
columns=['Name','Status','Type','Location','Description','FundingSource', 'ProjectTeam', 'ProjectTeamInfo','Image']
df = pd.DataFrame(data_list, columns=columns)

In [20]:
df.to_csv('berk_construction_june.csv',index=False,encoding='utf-8-sig')

### learn more abt bond revenue measures

In [15]:
bond_url = 'https://berkeleyca.gov/your-government/our-work/bond-revenue-measures'

#### scrape levelset site

In [2]:
base_url = 'https://www.levelset.com/projects/california/berkeley/'

In [4]:
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
driver.get(base_url)

[WDM] - Downloading: 100%|██████████| 7.70M/7.70M [00:01<00:00, 5.06MB/s]


In [7]:
links = []
for i in range(1,12):
    print(i)
    driver.get(base_url + '?page=' + str(i))
    project_cells = driver.find_elements(By.CLASS_NAME, 'card-body')
    # append to list of links
    links = links + [cell.find_element(By.TAG_NAME, 'h4').find_element(By.TAG_NAME,'a').get_attribute('href') for cell in project_cells]
    # go to next page
links[-5:]

1
2
3
4
5
6
7
8
9
10
11


['https://www.levelset.com/projects/california/berkeley/2065-kittredge-st-berkeley-ca-94704/',
 'https://www.levelset.com/projects/california/berkeley/2150-shattuck-ave-ste-110-berkeley-ca-94704/',
 'https://www.levelset.com/projects/california/berkeley/1042-ashby-ave-berkeley-ca-94710/',
 'https://www.levelset.com/projects/california/berkeley/2000-2nd-st-berkeley-ca-94710/',
 'https://www.levelset.com/projects/california/berkeley/910-ensenada-ave-berkeley-ca-94707/']

In [30]:
# for each link in list get following data
# name, company, status, type, image, description...
data_list = []
for i, link in enumerate(links):
    if i % 10 == 0:
        print(i, link)
    # go to page
    try:
        page = requests.get(link)
        soup = BeautifulSoup(page.content, "html.parser")

        # get info
        first_row = soup.find('div',class_='row')
        location = first_row.find('h1').text
        status = first_row.find('ul')
        project_start_date = status.find(string=re.compile("First known event: "))
        project_start_date = project_start_date.parent.text.split(': ')[1] if project_start_date else None
        project_update_date = status.find(string=re.compile("Last known event: "))
        project_update_date = project_update_date.parent.text.split(': ')[1] if project_update_date else None
        cards = soup.find_all('div',class_='card-body')
        type_of_construction = cards[0].find(string=re.compile("PROJECT TYPE")).parent.next_sibling.text 
        owner = cards[0].find(string=re.compile("PROPERTY OWNER")).parent.next_sibling.text 
        description = cards[1].find('p').text.strip()
        # print(type_of_construction, owner, project_start_date, project_update_date)
        # print(description)
    
        # add to row for project
        project_row = [location,project_start_date,project_update_date,type_of_construction,description,owner]
        data_list.append(project_row)
    except Exception as e:
        print('err',i, link, e)
        break
    
data_list[:2]

0 https://www.levelset.com/projects/california/berkeley/2220-oxford-st-berkeley-ca-94704/
10 https://www.levelset.com/projects/california/berkeley/2400-bancroft-way-berkeley-ca-94704/
20 https://www.levelset.com/projects/california/berkeley/2120-university-ave-berkeley-ca-94704/
30 https://www.levelset.com/projects/california/berkeley/2631-webster-st-berkeley-ca-94705/
40 https://www.levelset.com/projects/california/berkeley/2124-bancroft-way-berkeley-ca-94704/
50 https://www.levelset.com/projects/california/berkeley/2600-shattuck-ave-berkeley-ca-94704/
60 https://www.levelset.com/projects/california/berkeley/2080-addison-st-berkeley-ca-94704/
70 https://www.levelset.com/projects/california/berkeley/2748-san-pablo-ave-berkeley-ca-94702/
80 https://www.levelset.com/projects/california/berkeley/2111-university-ave-berkeley-ca-94704/
90 https://www.levelset.com/projects/california/berkeley/2067-university-ave-berkeley-ca-94704/
100 https://www.levelset.com/projects/california/berkeley/122

[['2220 Oxford St',
  '04/25/2018',
  '10/05/2021',
  'Commercial',
  'On this project at 2220 Oxford St, Berkeley, CA 94704 there have been 0 permits filed, 3 preliminary notices  exchanged, 1 lien waiver exchanged between companies and 0 liens filed. Below you can find when the various project and payment events occurred over the last several years of data where available. You can also report a payment event if you’re on this project.',
  'Resources for Community Development'],
 ['2929 7th St',
  '08/19/2019',
  '01/21/2022',
  'Commercial',
  'On this project at 2929 7th St, Berkeley, CA 94710 there have been 0 permits filed, 2 preliminary notices  exchanged, 0 lien waivers exchanged between companies and 0 liens filed. Below you can find when the various project and payment events occurred over the last several years of data where available. You can also report a payment event if you’re on this project.',
  '2929 Seventh Street LLC']]

In [31]:
columns=['Location','StartDate','LastUpdate','Type','Description','Owner']
df = pd.DataFrame(data_list, columns=columns)
df.to_csv('berk_construction_levelset.csv',index=False,encoding='utf-8-sig')

#### Combine Levelset and capital datasets

In [24]:
capital = pd.read_csv('berk_construction_june.csv')
level = pd.read_csv('berk_construction_levelset.csv')

In [25]:
level.head(2)

,Location,StartDate,LastUpdate,Type,Description,Owner
0,2220 Oxford St,04/25/2018,10/05/2021,Commercial,"On this project at 2220 Oxford St, Berkeley, C...",Resources for Community Development
1,2929 7th St,08/19/2019,01/21/2022,Commercial,"On this project at 2929 7th St, Berkeley, CA 9...",2929 Seventh Street LLC


In [26]:
capital.head(2)

,Name,StatusApril,Status,Type,Location,Description,FundingSource,ProjectTeam,ProjectTeamInfo,Image,Link,Center
0,1740 San Pablo Avenue,Design,Design,Affordable Housing,1740 San Pablo Avenue,Transform unutilized space at 1740 San Pablo i...,['Measure O'],"['Jenny Wyant', 'Bridge Housing']","[['Jenny Wyant', 'Senior Community Development...",https://berkeleyca.gov//sites/default/files/20...,https://berkeleyca.gov/your-government/our-wor...,NaN
1,1947 Center Street Facility Improvements,Design,Design,Facility,1947 Center Street,This is a Measure T1 Phase 2 project. The 1947...,['Measure T1'],['Elmar Kapfer '],"[['Elmar Kapfer ', 'Supervising ', 'Public Wor...",https://berkeleyca.gov//sites/default/files/20...,https://berkeleyca.gov/your-government/our-wor...,NaN


In [27]:
# create name column for levelset (same as location)
level.insert(loc=0,column='Name',value=level.Location)
level.insert(loc=5,column='Type_Level',value=level.Type)
level.head(2)

,Name,Location,StartDate,LastUpdate,Type,Type_Level,Description,Owner
0,2220 Oxford St,2220 Oxford St,04/25/2018,10/05/2021,Commercial,Commercial,"On this project at 2220 Oxford St, Berkeley, C...",Resources for Community Development
1,2929 7th St,2929 7th St,08/19/2019,01/21/2022,Commercial,Commercial,"On this project at 2929 7th St, Berkeley, CA 9...",2929 Seventh Street LLC


In [28]:
# match capital columns to levelset columns
capital_condensed = capital.loc[:,['Name','Location','Type','Description']]
capital_condensed['Owner'] = 'Berkeley'
capital_condensed.insert(2,'StartDate',None)
capital_condensed.insert(3,'LastUpdate',None)
capital_condensed.head(2)

,Name,Location,StartDate,LastUpdate,Type,Description,Owner
0,1740 San Pablo Avenue,1740 San Pablo Avenue,None,None,Affordable Housing,Transform unutilized space at 1740 San Pablo i...,Berkeley
1,1947 Center Street Facility Improvements,1947 Center Street,None,None,Facility,This is a Measure T1 Phase 2 project. The 1947...,Berkeley


In [14]:
level.Type.unique()

array(['Commercial', 'State / County', 'Owner Occupied Residential',
       'Residential', 'Federal', nan, 'Other'], dtype=object)

In [12]:
capital_condensed.Type.unique()

array(['Affordable Housing', 'Facility', 'Transportation',
       'Other Infrastructure', 'Sidewalk', 'Street', 'Park', 'Storm',
       'Sewer'], dtype=object)

In [15]:
constr_type_map_dict = {
    'Affordable Housing':'Residential',
    'Facility': 'State / County',
    'Transportation': 'State / County',
    'Other Infrastructure': 'State / County',
    'Sidewalk': 'State / County',
    'Street': 'State / County',
    'Park': 'State / County',
    'Storm': 'State / County',
    'Sewer': 'State / County'
}

In [29]:
# convert capital type to levelset type for comparison
capital_condensed.insert(5,'Type_Level',capital_condensed.Type.map(constr_type_map_dict))
capital_condensed.head(2)

,Name,Location,StartDate,LastUpdate,Type,Type_Level,Description,Owner
0,1740 San Pablo Avenue,1740 San Pablo Avenue,None,None,Affordable Housing,Residential,Transform unutilized space at 1740 San Pablo i...,Berkeley
1,1947 Center Street Facility Improvements,1947 Center Street,None,None,Facility,State / County,This is a Measure T1 Phase 2 project. The 1947...,Berkeley


In [30]:
constr_combined = pd.concat([level,capital_condensed])
print(level.shape, capital_condensed.shape, constr_combined.shape)

(271, 8) (105, 8) (376, 8)


In [23]:
constr_combined.to_csv('berk_construction_combined.csv',index=False)